<a href="https://colab.research.google.com/github/VNonTOP-DE/Learn-Spark/blob/main/Learn_spark_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('practice').getOrCreate()
spark

In [7]:
training = spark.read.csv('walmart.csv',header=True,inferSchema=True)

In [7]:
training.show()

+-------+----+--------+------------+--------+--------------------+--------------------+-------------+----------------+--------------------+--------------------+-----+------------+
|keyword|page|position|          id|    type|                name|               brand|averageRating|manufacturerName|    shortDescription|        thumbnailUrl|price|currencyUnit|
+-------+----+--------+------------+--------+--------------------+--------------------+-------------+----------------+--------------------+--------------------+-----+------------+
|perfume|   1|       1|1RQ2KVWAW4L5|Perfumes|Versace Bright Cr...|             Versace|          4.1|            NULL|Well known, Versa...|https://i5.walmar...|32.49|         USD|
|perfume|   4|      41|671UM03HM2MS|Perfumes|Jimmy Choo Eau De...|          Jimmy Choo|          4.5|            NULL|jimmy choo has cr...|https://i5.walmar...|32.97|         USD|
|perfume|   3|      37|4KRY0RE37RED|Perfumes|Emporio Armani St...|      Emporio Armani|          4.7

In [8]:
training.printSchema()

root
 |-- keyword: string (nullable = true)
 |-- page: string (nullable = true)
 |-- position: string (nullable = true)
 |-- id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- averageRating: string (nullable = true)
 |-- manufacturerName: string (nullable = true)
 |-- shortDescription: string (nullable = true)
 |-- thumbnailUrl: string (nullable = true)
 |-- price: string (nullable = true)
 |-- currencyUnit: string (nullable = true)



In [8]:
from pyspark.sql.types import DoubleType

# Cast 'averageRating' and 'price' to DoubleType
training = training.withColumn("averageRating", training["averageRating"].cast(DoubleType()))
training = training.withColumn("price", training["price"].cast(DoubleType()))
training = training.withColumn("position", training["position"].cast(DoubleType()))

In [9]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["averageRating","price"],outputCol="Independent Features")

In [10]:
output = featureassembler.transform(training)

In [14]:
output.show()

+-------+----+--------+------------+--------+--------------------+--------------------+-------------+----------------+--------------------+--------------------+-----+------------+--------------------+
|keyword|page|position|          id|    type|                name|               brand|averageRating|manufacturerName|    shortDescription|        thumbnailUrl|price|currencyUnit|Independent Features|
+-------+----+--------+------------+--------+--------------------+--------------------+-------------+----------------+--------------------+--------------------+-----+------------+--------------------+
|perfume|   1|       1|1RQ2KVWAW4L5|Perfumes|Versace Bright Cr...|             Versace|          4.1|            NULL|Well known, Versa...|https://i5.walmar...|32.49|         USD|         [4.1,32.49]|
|perfume|   4|      41|671UM03HM2MS|Perfumes|Jimmy Choo Eau De...|          Jimmy Choo|          4.5|            NULL|jimmy choo has cr...|https://i5.walmar...|32.97|         USD|         [4.5,32.

In [11]:
finalized_data = output.select("Independent Features","position")

In [18]:
finalized_data.show()

+--------------------+--------+
|Independent Features|position|
+--------------------+--------+
|         [4.1,32.49]|       1|
|         [4.5,32.97]|      41|
|         [4.7,87.95]|      37|
|          [4.7,54.6]|      34|
|          [4.7,25.3]|      33|
|          [4.6,20.0]|      32|
|         [4.3,33.31]|      31|
|         [4.4,13.11]|      30|
|         [4.5,43.49]|      29|
|          [4.4,33.0]|      28|
|         [4.3,13.98]|      27|
|          [4.6,24.4]|      26|
|          [4.6,32.0]|      25|
|         [4.7,76.43]|      24|
|         [4.5,54.99]|      23|
|          [3.7,78.0]|      22|
|         [4.6,41.99]|      21|
|         [4.6,35.87]|      20|
|         [4.9,74.99]|      19|
|         [4.4,25.98]|      18|
+--------------------+--------+
only showing top 20 rows



In [13]:
from pyspark.ml.regression import LinearRegression
training = training.filter(training["averageRating"].isNotNull() & training["price"].isNotNull())

train_data,test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='Independent Features', labelCol='position')
regressor = regressor.fit(train_data)

Py4JJavaError: An error occurred while calling o163.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 3) (06c8998d6d01 executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`VectorAssembler$$Lambda$3146/0x000000084134e840`: (struct<averageRating:double,price:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.sort_addToSorter_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unkn

In [14]:
from pyspark.ml.feature import VectorAssembler

# Create a VectorAssembler with handleInvalid set to "skip"
featureassembler = VectorAssembler(inputCols=["averageRating", "price"], outputCol="Independent Features", handleInvalid="skip")

# Transform the training data
output = featureassembler.transform(training)

# Select the features and target variable
finalized_data = output.select("Independent Features", "position")

# Split the data into training and testing sets
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])

# Create and train the Linear Regression model
regressor = LinearRegression(featuresCol='Independent Features', labelCol='position')
regressor = regressor.fit(train_data)

In [15]:
from pyspark.ml.regression import LinearRegression
training = training.filter(training["averageRating"].isNotNull() & training["price"].isNotNull())

train_data,test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='Independent Features', labelCol='position')
regressor = regressor.fit(train_data)

In [16]:
regressor.coefficients

DenseVector([-5.2518, 0.0412])

In [17]:
regressor.intercept

44.02676368951738

In [18]:
pred_results = regressor.evaluate(test_data)

In [19]:
pred_results.predictions.show()

+--------------------+--------+------------------+
|Independent Features|position|        prediction|
+--------------------+--------+------------------+
|          [3.0,98.0]|    37.0| 32.30833335325852|
|         [3.3,64.92]|     1.0|29.370129455019022|
|         [3.5,29.95]|     4.0| 26.87924909263721|
|         [3.7,48.99]|    42.0|26.613208989065583|
|          [3.7,76.9]|     6.0|27.762908889517213|
|          [3.7,78.0]|    22.0| 27.80822131841355|
|         [4.3,35.62]|     9.0|22.911385848439032|
|         [4.3,39.87]|     8.0|23.086456596447608|
|         [4.4,13.11]|    30.0|21.458950253426217|
|         [4.4,31.21]|     6.0|22.204545674356865|
|          [4.4,33.0]|    28.0|22.278281354106362|
|         [4.4,54.95]|    26.0|23.182470276174193|
|         [4.4,62.99]|    39.0|23.513662938289247|
|          [4.5,7.98]|     2.0|20.722451034975542|
|          [4.5,9.98]|     4.0| 20.80483726933252|
|         [4.5,14.75]|    14.0|21.001328438273912|
|         [4.5,18.27]|    16.0|

In [20]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(11.021838615881787, 169.99350570118298)